In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib qt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Check the aircraft

In [ ]:
from Vehicles.Conceptual_Planes.embraer_E190 import embraer_E190
from Vehicles.Conceptual_Planes.boeing_737_100 import boeing_737_100

Since we have set all the data, there are no missing parameters

In [ ]:
embraer_E190.CL_MAX

In [ ]:
embraer_E190.get_missing_parameters()

This is the Matching plot

In [ ]:
embraer_E190.WEIGHT_RATIO = 0.86

In [ ]:
res = embraer_E190.far_criteria(plot=True, start_plot=True, show_plot=True, clip=[1,150])

# Optimization

In [ ]:
def deviation_from_actual(
    plane,
    landing_curve,
    failed_approach_curve,
    takeoff_curve,
    climb_curve,
    cruise_curve,
    est_weight,
    est_area,
    est_thrust,
    OP_THRUST_LOADING,
    OP_WING_LOADING,
):
    AREA: float  = plane.AREA # 995.983667 ft^2
    MTOW : float = plane.MTOW # 105358.78 lb
    THRUST: float  = plane.THRUST # 40720 lb
        
    flag: float= 1
    if plane.MTOW/plane.AREA > landing_curve[0][0]:
        flag *= 10
        print("WARNING: MTOW/AREA")
    deviation = flag*(
        np.abs((est_weight- MTOW)/MTOW) + 
        np.abs((est_area-AREA)/AREA) + 
        np.abs((est_thrust-THRUST)/THRUST)
    )
    if plane.RANGE < 1000:
        flag *= 1000
    print(f"ARGS: {plane.ASPECT_RATIO=}, {plane.WEIGHT_RATIO=}, {plane.CL_MAX=}, \n{deviation=}")
    # print the takeoff curve slope 
    print(f"-------------------------")
    print(f"Wing Loading= {OP_WING_LOADING}, Thrust Loading = {OP_THRUST_LOADING}")
    print(f"-------------------------")
    print(f"{(est_weight-MTOW)/MTOW=},  {(est_area-AREA)/AREA=} ,  {(est_thrust-THRUST)/THRUST=}")
    print(f"-------------------------")
    print(f"{est_thrust=}, {THRUST=}")
    print(f"-------------------------")
    print(f"{est_area=}, {AREA=}")
    print(f"-------------------------")
    print(f"{est_weight=}, {MTOW=}")
    print(f"-------------------------")
    print(f"{plane.RANGE=}")

    return deviation 


In [ ]:
min_fun = embraer_E190.partial_fun_factory(
    deviation_from_actual,
    ['ASPECT_RATIO', "CL_MAX", "WEIGHT_RATIO"],
    # plot=True, start_plot=True, show_plot=True, clip=[1,200]
)

In [ ]:
opt = minimize(
    min_fun,
    x0 = [9.2, 2.2 , 0.86],
    # bounds=[(6, 15), (1.5, 3.5), (0.6, 0.9)],
    method = 'Nelder-Mead',
    options = {'disp':True, 'maxiter':1000}
)

## Display the results

In [ ]:
min_fun(opt.x, plot=True, start_plot=True, show_plot=True, clip=[1,200])

In [ ]:
embraer_E190.MLW/ embraer_E190.MTOW